In [ ]:
from cannect import mount

mount(r"E:\\SVN")

In [ ]:
from cannect import DataBaseCAN

db = DataBaseCAN.Reader(src='')
# 특정 프로젝트(G-프로젝트 등)를 위한 DB를 사용할 경우 json 파일 형식의 데이터를 @src에 사용
# 미적용 시 SVN 내 최신 DB 사용
print(db)
print(db.source)       # json 파일 전체 경로
print(db.traceability) # 원본 DB 이름 (json 파일명)
print(db.revision)     # 원본 DB의 SVN Revision 및 로컬 재생성 횟수

In [ ]:
db_hev = db.mode(engine_spec="HEV")
db_ice = db.mode(engine_spec="ICE")
print(db_hev)
print(db_ice)

In [ ]:
# db.to_dbc(engine_spec="ICE", channel=1)                                     # CANDBDuplicationError 발생: 중복 ID
db.to_dbc(engine_spec="ICE", channel=1, Codeword="Cfg_CanFDSTDDB_C == 0")   # 중복 제거를 위한 사양 명시

In [ ]:
db_dev = db.to_developer_mode("HEV")
print(db_dev)

In [ ]:
print(db[db["ECU"] != "EMS"])                   # EMS 기준, 수신 제어기만 선택
print(db[db["ICE Channel"].str.contains("P")])  # ICE 사양의 P-CAN 메시지만 선택

In [ ]:
# spec = DataBaseCAN.Specification(db)
# spec.generate("사양서 Tutorial.docx")              # filename 필수 입력

In [ ]:
vc = DataBaseCAN.VersionControl()
print(vc.name)          # DB 파일 이름 (확장자 제외)
print(vc.filename)      # DB 파일 이름 (확장자 포함)
print(vc.revision)      # DB 파일 최신 SVN Revision
print(vc.filepath)      # DB 파일 경로 (전체 경로)

# 현재 SVN 최신 CAN Excel SPEC에 대한 json 파일이 없는 경우 .json 호출 시 오류 발생
# 오류 발생 시 .to_json() 메소드 실행 필요
# vc.to_json()
print(vc.json)          # .revision에 해당하는 json 파일(전체 경로)
print(vc.history)       # DB 파일 SVN 이력

In [ ]:
from cannect import AscetCAN

model = AscetCAN.ComDef(DataBaseCAN.Reader(), "HEV")
model.generate()

In [ ]:
model = AscetCAN.ComRx(DataBaseCAN.Reader(), "HEV")
model.generate()

In [ ]:
model = AscetCAN.ComDiag(DataBaseCAN.Reader(), "CanFDMCUD_HEV", "MCU_01_10ms", "MCU_02_10ms", "MCU_03_100ms")
model.generate()

In [1]:
from cannect import mount, IntegrationRequest
mount(r"E:\\SVN")

AUTO_COMMIT = False

# 대상 모델에 대한 객체 생성
ir = IntegrationRequest("ComDef_HEV", "ComRx_HEV", "CanFDMCUD_HEV", "CanFDMCUM_HEV", "LogIf_HEV", "CAND")

# 산출물 관리 경로 지정
ir.deliverables = r"C:\Users\Administrator\Downloads\IR-TEST"

# 대상 모델의 "변경 전" 모델 선택
# - mode: {'latest', 'previous', 'select'}
# - mode == "latest"  : 대상 모델 Commit 미완료 상태, 최신 revision을 "변경 전" 모델로 간주 (기본 값)
# - mode == "previous": 대상 모델 Commit 완료 상태, 직전 revision을 "변경 전" 모델로 간주
# - mode == "select"  : 대상 모델에 대한 revision을 사용자가 모두 선택(console)
ir.select_previous_svn_version(mode="previous")

# SDD 자동 업데이트 (잠재적 오류 내포)
# 사용자는 개발 내용(이력)만 입력, 그 외 버전 관리는 자동 수행
# .rtf(SDD NOTE 파일)의 형식에 따라 동작하지 않을 수 있으므로 반드시 사용자의 주의 필요
ir.update_sdd("사용자 comment")

# 모델 자동 Commit (SDD, DSM, Polyspace는 수동 Commit - 추후 개선 예정)
# SVN Commit은 되돌릴 수 없으므로 신중하게 사용
if AUTO_COMMIT:
    ir.copy_model_to_svn()
    ir.commit_all("사용자 메시지: 반드시 영문으로 작성")

# IR 시트 작성
ir.resolve_svn_version() # 모델, SDD, DSM, Polyspace 의 SVN 버전 기입
ir.resolve_sdd_version() # SDD Version (FunctionVersion) 기입
ir.compare_model(exclude_imported=False) # .select_previous_svn_version()이 수행된 경우 "변경 전", "변경 후" 모델 비교

print(ir)
ir.to_clipboard() # 이후 Excel IR 문서에 붙여넣기

2026-02-04 16:42:31 Updating 'E:\SVN\model':
At revision 23018.
2026-02-04 16:42:32 Updating 'E:\SVN\GSL_Build\7_Notes':
At revision 53851.
2026-02-04 16:42:32 Updating 'E:\SVN\GSL_Build\1_AswCode_SVN\PostAppSW\0_XML\DEM_Rename':
At revision 53851.
2026-02-04 16:42:32 Updating 'E:\SVN\Autron_CoWork\사이버보안\Module_Test_Results':
At revision 13611.
2026-02-04 16:42:32 [INITIALIZE]
2026-02-04 16:42:33 [UPDATE SDD NOTE AND SAVE TO'C:\Users\Administrator\Downloads\IR-TEST\05_Resources\SDD']
2026-02-04 16:42:33 >>> %ComDef_HEV   : ALREADY UPDATED
2026-02-04 16:42:33 >>> %ComRx_HEV    : ALREADY UPDATED
2026-02-04 16:42:33 >>> %CanFDMCUD_HEV: ALREADY UPDATED
2026-02-04 16:42:33 >>> %CanFDMCUM_HEV: ALREADY UPDATED
2026-02-04 16:42:33 >>> %LogIf_HEV    : ALREADY UPDATED
2026-02-04 16:42:33 >>> %CAND         : ALREADY UPDATED
2026-02-04 16:42:33 [RESOLVE SVN VERSION]
2026-02-04 16:42:34 >>> %ComDef_HEV    -> 22550 |                                     | 040g00002u801q070g7g807i9bh0a.zip -> 53076 | 

    FunctionName FunctionVersion  \
0     ComDef_HEV       00.00.008   
1      ComRx_HEV       00.00.008   
2  CanFDMCUD_HEV       00.05.004   
3  CanFDMCUM_HEV       00.05.003   
4      LogIf_HEV       00.05.019   
5           CAND       00.02.006   

                                             SCMName SCMRev  \
0  HNB_GASOLINE\_29_CommunicationVehicle\Standard...  22550   
1  HNB_GASOLINE\_29_CommunicationVehicle\Standard...  22550   
2  HNB_GASOLINE\_29_CommunicationVehicle\CANInter...  22936   
3  HNB_GASOLINE\_29_CommunicationVehicle\CANInter...  22431   
4  HNB_GASOLINE\_29_CommunicationVehicle\Communic...  22500   
5  HNB_GASOLINE\_30_CommunicationCoordinator\CANB...  23018   

                      DSMName DSMRev BSWName BSWRev  \
0                                                     
1                                                     
2  canfdmcud_hev_confdata.xml  51722                  
3                                                     
4      logif_hev_confdata.xml 

In [2]:
from cannect import ChangeHistoryManager

# ir 인스턴스가 존재한다고 가정
# ∵ ir = IntegrationRequest(*modeuls)
ppt = ChangeHistoryManager(ir)

# 1페이지 작성
ppt.title       = "[CAN/공통] CAN BUS-OFF Recover Post Run 할당"   # 제목
ppt.developer   = "이제혁"                                         # 개발 담당
ppt.issue       = "VCDM CR10786223"                               # Issue
ppt.lcr         = "자체 개선"                                      # LCR
ppt.problem     = "OTA 업데이트 평가 중, ... 발생"                   # 문제 현상

2026-02-04 16:42:48 [WRITE PPT ON "C:\Users\Administrator\Downloads\IR-TEST\0000_변경내역서 양식.pptx"]
2026-02-04 16:42:48 >>> RESIZING COVER PAGE
2026-02-04 16:42:49 >>> WRITE ON "SW 기능 상세 설명 - 변경 전"
2026-02-04 16:42:49 >>> WRITE ON "SW 기능 상세 설명 - 변경 후"
2026-02-04 16:42:49 >>> GENERATE SLIDES 
2026-02-04 16:42:51 >>> WRITING PREVIOUS MODEL DETAILS...
2026-02-04 16:42:51 >>> ... ComDef_HEV @22550
2026-02-04 16:42:51 >>> ... ComRx_HEV @22550
2026-02-04 16:42:52 >>> ... CanFDMCUD_HEV @22936
2026-02-04 16:42:52 >>> ... CanFDMCUM_HEV @22431
2026-02-04 16:42:53 >>> ... LogIf_HEV @22500
2026-02-04 16:42:53 >>> ... CAND @23018
2026-02-04 16:42:54 >>> WRITING REVISED MODEL DETAILS...
2026-02-04 16:42:54 >>> ... ComDef_HEV @22550
2026-02-04 16:42:55 >>> ... ComRx_HEV @22550
2026-02-04 16:42:56 >>> ... CanFDMCUD_HEV @22936
2026-02-04 16:42:57 >>> ... CanFDMCUM_HEV @22431
2026-02-04 16:42:58 >>> ... LogIf_HEV @22500
2026-02-04 16:42:59 >>> ... CAND @23018
